## Initial Setup

In [99]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\top100.json


In [100]:
df = pd.read_json('input/top100.json')
df.head()

,id,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
0,1,1,Mariah Carey,All I Want For Christmas Is You,1 Last Week,1 Peak Rank,37 Weeks on Chart,2020-01-04
1,2,2,Brenda Lee,Rockin' Around The Christmas Tree,2 Last Week,2 Peak Rank,32 Weeks on Chart,2020-01-04
2,3,3,Bobby Helms,Jingle Bell Rock,9 Last Week,3 Peak Rank,30 Weeks on Chart,2020-01-04
3,4,4,Burl Ives,A Holly Jolly Christmas,6 Last Week,4 Peak Rank,15 Weeks on Chart,2020-01-04
4,5,5,Post Malone,Circles,3 Last Week,1 Peak Rank,17 Weeks on Chart,2020-01-04


In [101]:
len(df.loc[df['rank_last_week'] == df['weeks_on_chart']]) == len(df)


False

In [102]:
df['rank_last_week'] = df['rank_last_week'].apply(lambda rank_last_week: rank_last_week[:-10])
df['peak_rank'] = df['peak_rank'].apply(lambda peak_rank: peak_rank[:-10])
df['weeks_on_chart'] = df['weeks_on_chart'].apply(lambda weeks_on_chart: weeks_on_chart[:-15])

In [103]:
df['date'].dtype

dtype('<M8[ns]')

In [104]:
unique_artists = df['artist'].unique()
print(unique_artists)
len(unique_artists)

['Mariah Carey' 'Brenda Lee' 'Bobby Helms' 'Burl Ives' 'Post Malone'
 'Arizona Zervas' 'Andy Williams' 'Lewis Capaldi' 'Maroon 5' 'Lizzo'
 'Wham!' 'Jose Feliciano' 'Roddy Ricch' 'Tones And I' 'Dean Martin'
 'Nat King Cole' 'Dan + Shay & Justin Bieber' 'Selena Gomez'
 'Mustard & Roddy Ricch' 'DaBaby' 'The Ronettes' 'Gene Autry'
 'Billie Eilish' 'Chris Brown Featuring Drake'
 'Shawn Mendes & Camila Cabello' 'Juice WRLD & YoungBoy Never Broke Again'
 'Perry Como And The Fontane Sisters With Mitchell Ayres And His Orchestra'
 'Darlene Love' 'Lil Nas X' 'Kelly Clarkson' 'Young Thug Featuring Gunna'
 'Lil Nas X Featuring Billy Ray Cyrus' 'Chuck Berry' 'Perry Como'
 'Travis Scott' 'The Weeknd' 'Elvis Presley' 'Bing Crosby' 'Frank Sinatra'
 'Jonas Brothers' 'SHAED' 'Lil Baby' 'Lil Uzi Vert' 'Thurl Ravenscroft'
 'Lil Tecca' 'Trevor Daniel' 'Dua Lipa' 'Harry Styles' 'Old Dominion'
 'blackbear' 'YNW Melly & Juice WRLD' 'Taylor Swift'
 'Wale Featuring Jeremih' 'Luke Combs' 'Don Toliver' 'Halsey'
 

451

In [105]:
savage_data = df.loc[df['artist'].str.find('21 Savage') >= 0]
savage_data

,id,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
1325,1326,26,Lil Uzi Vert & 21 Savage,Yessirskiii,-,26,1,2020-04-04
1492,1493,93,Lil Uzi Vert & 21 Savage,Yessirskiii,26,26,2,2020-04-11
4208,4209,9,21 Savage & Metro Boomin,Runnin,-,9,1,2020-10-24
4209,4210,10,21 Savage & Metro Boomin Featuring Drake,Mr. Right Now,-,10,1,2020-10-24
4218,4219,19,21 Savage & Metro Boomin,Glock In My Lap,-,19,1,2020-10-24
4225,4226,26,21 Savage & Metro Boomin Featuring Young Thug,Rich N*gga Sh*t,-,26,1,2020-10-24
4231,4232,32,21 Savage & Metro Boomin,Slidin,-,32,1,2020-10-24
4232,4233,33,21 Savage & Metro Boomin,Many Men,-,33,1,2020-10-24
4255,4256,56,21 Savage & Metro Boomin,My Dawg,-,56,1,2020-10-24
4256,4257,57,21 Savage & Metro Boomin,Brand New Draco,-,57,1,2020-10-24


In [106]:
savage_data.describe()

,id,rank
count,33.000000,33.000000
mean,4251.030303,57.090909
std,765.189784,25.167394
min,1326.000000,9.000000
25%,4256.000000,36.000000
50%,4336.000000,57.000000
75%,4549.000000,76.000000
max,4965.000000,94.000000
